# Project to Explore a sales figures data series

The objective of this notebook is to have a project to play around with, that is practical and can get me some experience in the area and serve as curriculum.

Questions to do in terms of EDA and data preparation:
-General importing, null removal, general cleaning analysis, basic graphical representation, separation of it's 3 logical classes, transformation of these said tables into usable classes and a small non SQL database.

Questions in general regarding to the data:

1. Can we make a preditive analyisis of the time between Order and ship dates based upon the caracteristics of an order?
2. Creating a predictive sales model for the company's sales (maybe divided by state and or product range or line).
3. Make a costumer analyis: which are our best costumeres that we should target more? How can we increase the average spending per order?
4. Make a product line analysis, what are our winner products? What should we bet on? ABC analysis? what to (maybe) discontinue?



Small roadmap for the bros out there on the cleaning and importing that i did:
I based myself on the work of Samruddhi Mhatre https://github.com/SamruddhiMhatre 

When doing all of this, this cheat-sheet will be extremly usefull: https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf

-**import**  

-**convert to the correct data types**  Be carefull with dates and currencies. Be open to use open costum functions to do it if the data types are a bit hairy.  

-**Check for null values and handle them**

-**Use the describe and head fucntions to understand what's up in general**

-**Divide the data into logical sub-tables if you find it necessary**



Importing all necessary modules:


In [2]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
sales_data_simple = pd.read_csv("sales_time_series.csv",index_col=[0])


In [12]:
#sales_data = pd.read_csv("sales_time_series.csv",index_col=[0])

#this is an example of how to have things well converted, already since the begining

conversion_dictionary = {"Ship Mode": "string",
                         "Order ID" : "string",
                         "Customer ID" : "string",
                         "Customer Name" : "string",
                         "Segment" : "string",
                         "City" : "string",
                         "State" : "string",
                         "Country" : "string",
                         #"Postal Code" : "int", <- there are some null values here in which where not interested
                         "Region" : "string",
                         "Product ID" : "string",
                         "Category" : "string",
                         "Sub-Category" : "string",
                         "Product Name" : "string"}

sales_data = pd.read_csv("sales_time_series.csv",
                         index_col=[0],
                         dtype=conversion_dictionary)

#It's also necessary to convert the dates and the currencies
sales_data["Order Date"] = pd.to_datetime(sales_data["Order Date"])
sales_data["Ship Date"] = pd.to_datetime(sales_data["Ship Date"])

Interesting notes: to see function definitions just press on the function pressin shift and tab 2 times. This will giv eall of the functions's input.
We might also want to use our own self made functions to make more compelx transformations a bit easier.
In the future i should look into lambda functions.

In [13]:
def convert_id(id_value):
    new_value = id_value.replace('-',' ')
    return str(new_value)

#sales_data["Order ID"] = sales_data["Order ID"].apply(convert_id) 

#<- In this way, there really isn't any need to use parethesis 
#as the function assumes the input as the panda entry

In [19]:
print(sales_data.isnull().sum()) #<- check for nulls

sales_data[sales_data["Postal Code"].isnull()] #<- shows the actual null values


Order ID         0
Order Date       0
Ship Date        0
Ship Mode        0
Customer ID      0
Customer Name    0
Segment          0
Country          0
City             0
State            0
Postal Code      0
Region           0
Product ID       0
Category         0
Sub-Category     0
Product Name     0
Sales            0
dtype: int64


,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
Row ID,,,,,,,,,,,,,,,,,


It seems that the problem is always with the town of Burlington Vermont. As it isn't very hard to correct this, let's palce the correct values.

In [18]:
sales_data["Postal Code"] = sales_data["Postal Code"].fillna(5401)

The null problem is now corrected. Let's we what's generally happening with the data then. Let's also order it by date of order.

In [35]:
sales_data.head()
sales_data.sort_values("Order Date")
#sales_data["Sales"].plot()


,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
Row ID,,,,,,,,,,,,,,,,,
541,CA-2015-140795,2015-01-02,2015-03-02,First Class,BD-11500,Bradley Drucker,Consumer,United States,Green Bay,Wisconsin,54302.0,Central,TEC-AC-10001432,Technology,Accessories,Enermax Aurora Lite Keyboard,468.900
5714,US-2015-143707,2015-01-03,2015-05-03,Standard Class,HR-14770,Hallie Redmond,Home Office,United States,New York City,New York,10035.0,East,TEC-PH-10003655,Technology,Phones,Sannysis Cute Owl Design Soft Skin Case Cover ...,5.940
158,CA-2015-104269,2015-01-03,2015-06-03,Second Class,DB-13060,Dave Brooks,Consumer,United States,Seattle,Washington,98115.0,West,FUR-CH-10004063,Furniture,Chairs,Global Deluxe High-Back Manager's Chair,457.568
6549,CA-2015-113880,2015-01-03,2015-05-03,Standard Class,VF-21715,Vicky Freymann,Home Office,United States,Elmhurst,Illinois,60126.0,Central,OFF-PA-10003036,Office Supplies,Paper,"Black Print Carbonless 8 1/2"" x 8 1/4"" Rapid M...",17.472
7950,CA-2015-131009,2015-01-03,2015-05-03,Standard Class,SC-20380,Shahid Collister,Consumer,United States,El Paso,Texas,79907.0,Central,FUR-FU-10001095,Furniture,Furnishings,DAX Black Cherry Wood-Tone Poster Frame,63.552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
909,CA-2018-143259,2018-12-30,2019-03-01,Standard Class,PO-18865,Patrick O'Donnell,Consumer,United States,New York City,New York,10009.0,East,OFF-BI-10003684,Office Supplies,Binders,Wilson Jones Legal Size Ring Binders,52.776
646,CA-2018-126221,2018-12-30,2019-05-01,Standard Class,CC-12430,Chuck Clark,Home Office,United States,Columbus,Indiana,47201.0,Central,OFF-AP-10002457,Office Supplies,Appliances,Eureka The Boss Plus 12-Amp Hard Box Upright V...,209.300
908,CA-2018-143259,2018-12-30,2019-03-01,Standard Class,PO-18865,Patrick O'Donnell,Consumer,United States,New York City,New York,10009.0,East,TEC-PH-10004774,Technology,Phones,Gear Head AU3700S Headset,90.930


In [ ]:
sales_data["Sales"].describe()

In [20]:
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9800 entries, 1 to 9800
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Order ID       9800 non-null   string        
 1   Order Date     9800 non-null   datetime64[ns]
 2   Ship Date      9800 non-null   datetime64[ns]
 3   Ship Mode      9800 non-null   string        
 4   Customer ID    9800 non-null   string        
 5   Customer Name  9800 non-null   string        
 6   Segment        9800 non-null   string        
 7   Country        9800 non-null   string        
 8   City           9800 non-null   string        
 9   State          9800 non-null   string        
 10  Postal Code    9800 non-null   float64       
 11  Region         9800 non-null   string        
 12  Product ID     9800 non-null   string        
 13  Category       9800 non-null   string        
 14  Sub-Category   9800 non-null   string        
 15  Product Name   9800 n

By the way the data is structured, i suspect that orders contain more than 1 product. Let's check it by testing uniqueness of values.

In [37]:
sales_data["Order ID"].value_counts()

CA-2018-100111    14
CA-2018-157987    12
US-2017-108504    11
CA-2017-165330    11
CA-2017-105732    10
                  ..
CA-2017-102596     1
US-2018-134481     1
CA-2017-157686     1
CA-2017-150077     1
CA-2016-142734     1
Name: Order ID, Length: 4922, dtype: Int64

Okay, several order ID's repeat themselves, let's try to understand why by isolating these lines.


In [38]:
sales_data[sales_data["Order ID"] == "CA-2018-157987"  ]

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
Row ID,,,,,,,,,,,,,,,,,
1947,CA-2018-157987,2018-02-09,2018-06-09,Standard Class,AC-10615,Ann Chong,Corporate,United States,New York City,New York,10009.0,East,OFF-AR-10000658,Office Supplies,Art,Newell 324,23.100
1948,CA-2018-157987,2018-02-09,2018-06-09,Standard Class,AC-10615,Ann Chong,Corporate,United States,New York City,New York,10009.0,East,FUR-FU-10001196,Furniture,Furnishings,DAX Cubicle Frames - 8x10,11.540
1949,CA-2018-157987,2018-02-09,2018-06-09,Standard Class,AC-10615,Ann Chong,Corporate,United States,New York City,New York,10009.0,East,FUR-TA-10001889,Furniture,Tables,Bush Advantage Collection Racetrack Conference...,254.526
1950,CA-2018-157987,2018-02-09,2018-06-09,Standard Class,AC-10615,Ann Chong,Corporate,United States,New York City,New York,10009.0,East,OFF-AP-10000358,Office Supplies,Appliances,Fellowes Basic Home/Office Series Surge Protec...,12.980
1951,CA-2018-157987,2018-02-09,2018-06-09,Standard Class,AC-10615,Ann Chong,Corporate,United States,New York City,New York,10009.0,East,OFF-BI-10004970,Office Supplies,Binders,"ACCOHIDE 3-Ring Binder, Blue, 1""",26.432
1952,CA-2018-157987,2018-02-09,2018-06-09,Standard Class,AC-10615,Ann Chong,Corporate,United States,New York City,New York,10009.0,East,TEC-PH-10003885,Technology,Phones,Cisco SPA508G,197.970
1953,CA-2018-157987,2018-02-09,2018-06-09,Standard Class,AC-10615,Ann Chong,Corporate,United States,New York City,New York,10009.0,East,OFF-LA-10001641,Office Supplies,Labels,Avery 518,18.900
1954,CA-2018-157987,2018-02-09,2018-06-09,Standard Class,AC-10615,Ann Chong,Corporate,United States,New York City,New York,10009.0,East,FUR-CH-10003379,Furniture,Chairs,Global Commerce Series High-Back Swivel/Tilt C...,1282.410
1955,CA-2018-157987,2018-02-09,2018-06-09,Standard Class,AC-10615,Ann Chong,Corporate,United States,New York City,New York,10009.0,East,OFF-AR-10004582,Office Supplies,Art,BIC Brite Liner Grip Highlighters,4.920


And it's confirmed. There are repeating values. The best thing to do now is to split the several components of the list.

Let's split the tables into atomized values.



In [42]:
orders_data = sales_data[["Order ID", "Order Date", "Ship Date", "Customer ID", "Product ID","Sales"]]
costumers_data = sales_data[["Customer ID", "Customer Name", "Segment", "Country","City","State", "Postal Code", "Region"]]
product_data = sales_data[["Product ID", "Category", "Sub-Category", "Product Name", "Sales"]]


In [43]:
orders_data.head()
costumers_data.head()
product_data.head()

,Product ID,Category,Sub-Category,Product Name,Sales
Row ID,,,,,
1,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
2,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
3,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
4,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
5,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680
